<h1 align="center">Parsing with Context Free Grammars(CFG) and PCFG (Probabilistic CFG)</h1> 

In [ ]:
import nltk
import io
import IPython
import PIL.Image
from nltk import Tree
from nltk.draw.util import CanvasFrame
from nltk.draw import TreeWidget
from IPython.display import Image
from IPython.display import display
from IPython.display import HTML
import pprint

- It is assumed that NLTK is already installed on the notebook.
- We need to install `punkt` tokeniser as well
- the code snippet `nltk.download('punkt')` should install the tokeniser
- If it does not work, type `nltk.download()`, a gui based window will appear.
    - Click on the tab `Model`
    - Selcect the option `punkt`
    - click on `Download`

In [ ]:
try:
    nltk.download('punkt')
except:
    nltk.download()

## Question 5, 6

In [ ]:

# Input addresses
f = ['773-702-7034 Carl Sandburg Middle School , W. Hawley Street , LOCATED IN THE GYM ,Mundelein, IL , us , 60060', 
'Harold Washington Library Center , 400 S. State Street , Cindy Pritzker Auditorium , Chicago , IL , us , 60605',
'Dive Chicago , Slip K-27 , Burnham Harbor , Chicago , IL , us , 60660',
'Kettle Moraine State Forest Southern Unit , Forest Headquarters , S91 W39091 Highway 59 , Eagle , WI , us , 53119',
'Jack Benny Park , Downtown Waukegan , Corner of Clayton and Genesee Streets , Waukegan , IL , us , 60085']


# Grammar G1
grammar1 = nltk.CFG.fromstring("""
S ->  N NM ADD1 ADD2 CT STATE COUNTRY Z |NM ADD1 ADD2 CT STATE COUNTRY Z| NM ADD1 CT STATE COUNTRY Z
NM -> 'NAME' NM | 'NAME' 'NUM' NM | 'NAME' 'NUM' | 'NAME'
ADD1 -> N DIR ST | ST | N DIR | N DIR AVE | AVE | N AVE | N ST |AVE N| ST N | AVE DIR |ST DIR | AVE DIR AVE| AVE DIR AVE |ST DIR ST |AVE N AVE | ST N ST | N DIR N SS AVE |N DIR N SS ST | N DIR N SS AVE |N DIR N SS ST |  DIR AVE |DIR ST
ADD2 -> FL ST1 | FL | SUI | SUI FL | SUI FL ST1 | FL SUI | DIR ST1 |N DIR ST1 | ST1 | N DIR | N ST1 | N SS ST1 | ST1 N |ST1 DIR ST1 | 'NUM'
N -> 'NUM' 
CT -> 'CITY' | 'CITY' 'CITY' | 'CITY' 'CITY' 'CITY'
STATE -> 'STATE' | STATE 'STATE'
Z -> 'ZIP'
COUNTRY -> 'COUNTRY'| COUNTRY 'COUNTRY'|'us'|'US'
DIR -> 'N.' | 'S.' | 'W.'|'E.'|'North'|'South'|'West'|'East'|'N'|'S'|'E'|'W'|'w.'|'s.'|'n.'|'e.'
ST -> 'STREET1' ST | 'STREET1' 'NUM' | 'STREET1' 'NUM' ST| 'STREET1'
AVE -> 'AVENUE' AVE | 'AVENUE'
ST1 -> 'STREET2' ST1 | 'STREET2'
SUI -> SU N
FL -> N SS F | N F | OTH F | F N SS
OTH -> 'Top' | 'Main'
F -> 'FLOOR'
SU -> 'SUITE'
SS -> 'SUP'
""")

pattern = [(r'^(?!(?:^[0-9][0-9][0-9][0-9][0-9]+$)$)\d+','NUM'),
           (r'^(?!(?:^[0-9][0-9][0-9][0-9][0-9]+$)|(\(|\)|us|US|N\.|W\.|E\.|S\.|N|W|E|S|n\.|w.|e\.|s\.|Ave|Ave\.|ave|avenue|Avenue|AVENUE|ave\.|Suite|suite|SUITE|Floor|Fl.|Fl|floor|nd|th)$).*','STREET1'),
           (r'(Suite|suite|SUITE)','SUITE'),(r'(nd|th)','SUP'),(r'(floor|Fl.|Fl|Floor)','FLOOR'),(',','COM'),
           (r'\(|\)','BRAC'),(r'Ave|Ave.|ave|avenue|Avenue|AVENUE|ave.','AVENUE'),(r'^[0-9][0-9][0-9][0-9][0-9]+$','ZIP')]

count = 0
total_count = 0
tagger = nltk.RegexpTagger(pattern)
parser = nltk.ChartParser(grammar1)

for line in f:
    c=0
    adr1 = line
    t=[]
    f=0
    p=0
    start=0
    end=0
    x = None
    
    for (y,x) in (tagger.tag(nltk.word_tokenize(adr1))):
        
        if y == ',':
            x = 'COM'

        if x is not None and x != 'COM' and f==0:
            if x == 'STREET1' :

                t.append('NAME')
            else:
                t.append(x)
        elif x is not None and x != 'COM' and f==1:
            t.append(x)
        elif x is not None and x != 'COM'  and f==2:
            if x == 'STREET1' :
                x='STREET2'
            t.append(x)
        elif x is not None and x != 'COM'  and f==3:
            if x == 'STREET1' :
                x='CITY'
            t.append(x)
        elif x is not None and x != 'COM' and f==4:
            if x == 'STREET1' :
                x='STATE'
            t.append(x)
        elif x is not None and x != 'COM' and  f==5:
            if x == 'STREET1' :
                x='COUNTRY'
            t.append(x)     
        elif x is not None and x != 'COM'  and f==6:
            t.append(x)
        elif x == 'COM' :
            f+=1
        elif x is None  :
            t.append(y)
    
    num_of_parse = len(list(parser.parse(t)))

    print num_of_parse,adr1
    for tree in parser.parse(t):
        if num_of_parse > 1:
            print tree

## Question 7, 8

In [ ]:
import math
#an example address string
adr1 = "Jack Benny Park , Downtown Waukegan , Corner of Clayton and Genesee Streets , Waukegan , IL , us , 60085"

#specify the grammer with each production with the probabilities
grammar1 = \
    nltk.PCFG.fromstring("""
S -> NM X1 [0.6] | NM X6 [0.4]
X6 -> ADD1 X3 [1.0]
X1 -> ADD1 X2 [1.0]
X2 -> ADD2 X3 [1.0]
X3 -> CT X4 [1.0]
X4 -> STATE X5 [1.0]
X5 -> COUNTRY Z [1.0]
NM -> X7 NM [0.4] | X7 X8 [0.1]  | 'NAME' [0.5] 
X7 -> 'NAME' [1.0]
X8 -> N NM [1.0]
ADD1 -> X9 ST [0.3] | ST STREET1 [0.25] | 'STREET1' [0.1] | N DIR [0.1] | X9 AVE [0.05] | AVENUE AVE [0.1]| 'AVENUE' [0.06]| N ST [0.04]
X9 -> N DIR [1.0]
ADD2 -> FL ST1 [0.1]| X11 F [0.033]| N F [0.067]| OTH F [0.1]| F X11 [0.1]| SU N [0.1]| SUI FL [0.1]| SUI X13 [0.05]| FL SUI [0.05]| DIR ST1 [0.02]| N X14 [0.04]| STREET2 ST1 [0.04]| 'STREET2' [0.01]| N DIR [0.04]| N ST1 [0.05]| N X15 [0.01]| ST1 N [0.02]|ST1 X16 [0.02]| 'NUM' [0.05]
X16 -> DIR ST1 [1.0]
X15 -> SS ST1 [1.0]
X14 -> DIR ST1 [1.0]
X13 -> FL ST1 [1.0]
N -> 'NUM' [1.0]
CITY -> 'CITY' [1.0]
CT -> 'CITY' [0.4] | CITY CITY [0.6]
STATE -> 'IL' [1.0]
Z -> 'ZIP' [1.0]
COUNTRY -> 'us' [0.7] | 'US' [0.3]
DIR -> 'N.' [0.1]| 'S.'[0.1] | 'W.' [0.1] |'E.' [0.1] |'North' [0.05] |'South' [0.05] |'West' [0.05]|'East' [0.05]|'N' [0.05] |'S' [0.05]|'E' [0.05] |'W' [0.05]|'w.' [0.05]|'s.' [0.05] |'n.' [0.05] |'e.' [0.05]
STREET1 -> 'STREET1' [1.0]
ST -> STREET1 ST [0.3] | STREET1 N [0.2]| STREET1 X10 [0.2]| 'STREET1' [0.3]
X10 -> N ST [1.0]
AVE -> AVENUE AVE [0.6] | 'AVENUE' [0.4]
AVENUE -> 'AVENUE' [1.0]
STREET2 ->	'STREET2' [1.0]
ST1 -> STREET2 ST1 [0.6] | 'STREET2' [0.4]
SUI -> SU N [1.0]
FL -> X11 F [0.3] | N F [0.2] | OTH F [0.2] | F X11 [0.3]
X11 -> N SS [1.0]
OTH -> 'Top' [0.5] | 'Main' [0.5]
F -> 'FLOOR' [1.0]
SU -> 'SUITE' [1.0]
SS -> 'SUP' [1.0]
""")

pattern = [(r'^(?!(?:^6[0-9]{4,4}$)$)\d+','NUM'),(r'^(?!(?:^6[0-9]{4,4}$)|(\(|\)|IL|us|US|N\.|W\.|E\.|S\.|N|W|E|S|n\.|w\.|e\.|s\.|Ave|Ave\.|ave|avenue|Avenue|AVENUE|ave\.|West|North|East|,|South|Suite|suite|SUITE|Floor|Fl.|Fl|floor|nd|th)$).*','STREET1'),(r'(Suite|suite|SUITE)','SUITE'),(r'(nd|th)','SUP'),(r'(floor|Fl.|Fl|Floor)','FLOOR'),(r',','COM'),(r'\(|\)','BRAC'),(r'Ave|Ave.|ave|avenue|Avenue|AVENUE|ave.','AVENUE'),(r'^6[0-9]{4,4}$','ZIP')]
tagger = nltk.RegexpTagger(pattern)
    
t = []
f = 0
p = 0
start = 0
end = 0
for (y, x) in tagger.tag(nltk.word_tokenize(adr1)):
    if x is not None and x != 'COM' and x != 'BRAC' and f == 0:
        if x == 'STREET1':
            x = 'NAME'
        t.append(x)
    elif x != None and x != 'COM' and x != 'BRAC' and f == 1:
        t.append(x)
    elif x != None and x != 'COM' and x != 'BRAC' and f == 2:
        if x == 'STREET1':
            x = 'STREET2'
        t.append(x)
    elif x != None and x != 'COM' and x != 'BRAC' and f == 3:
        if x == 'STREET1':
            x = 'CITY'
        t.append(x)
    elif x != None and x != 'COM' and x != 'BRAC' and f == 6:
        t.append(x)
    elif x == 'COM':
        f += 1
    elif x == None:
        t.append(y)
            
tokens = t
grammar = grammar1


    
#creating table
size = len(tokens)
table = [ [[] for i in range(size)] for i in range(size) ]

viterbi_parser = nltk.ViterbiParser(grammar) 

for tree in viterbi_parser.parse(tokens):
    print tree



In [ ]:
from parser import *

#sentence
s = 'a boy with a telescope saw a girl'
parser = PCFGParser('model.bin') 
parser.writeOutResult(s, '2.out')
result, value = parser.parse(s)
# Grammar Productions
print parser.unaries
print parser.binaries

print result
print value